# Chap06 连接

In [1]:
import pandas as pd
import numpy as np

## 关系型连接
### 值连接`merge`
1. `df1.merge(df2)`
   - `on`两个表的连接的列，列名必须一样
   - `how`连接的方式，有以下几种：`left/right/inner/outer`
   - `left_on、right_on`两个表中需要连接的列不具备相同的列名时，则通过这两个参数指定
   - `suffixes`两个表中出现了重复的列名时，通过该参数可以对重复的列名进行区分
2. `merge`中提供了`validate`参数检查连接的唯一性模式
### 索引连接`join`
1. 把索引当作键进行连接
2. `df1.join(df2)`
   - `on`指索引名，单层索引时省略参数表示按照当前索引连接
   - `how`连接的方式
   - `lsuffix、rsuffix`对重复的列指定左右后缀
### 方向连接`concat`
1. 实现两个表或者多个表按照纵向或者横向拼接，本质上还是关于索引进行连接
   - 纵向拼接会根据列索引对齐
   - 横向拼接会根据行索引对齐
2. `pd.concat([df1,df2])`
   - `axis`表示拼接方向
     - 默认状态`axis=0`表示纵向拼接多个表，常用于多个样本的拼接
     - `axis=1`表示横向拼接多个表，常用于多个字段或特征的拼接
     - 可以先用`reset_index`方法恢复默认整数索引在进行合并
   - `join`
     - 默认状态下`join=outer`，表示保留所有的列，并将不存在的值设为缺失
     - `join=inner`，表示保留两个表都出现过的列
   - `keys`用于多表合并后，标记新表的数据来源于哪个原表
### 序列与表的合并`append/assign`
1. `append`把一个序列追加到表的行末
   - `df1.append(s)`原表如果是默认整数序列的索引，可以使用`ignore_index=True`对新序列对应的索引自动标号，否则必须对Series指定name属性
2. `assign`添加新的列
   - `df['new_col'] = ...`等价地添加新列，但会直接在原表上进行改动
   - assign返回的是一个临时副本

In [3]:
df1 = pd.DataFrame({'Name': ['San Zhang','Si Li'],'Age':[20,30]})
df2 = pd.DataFrame({'Name':['Si Li','Wu Wang'], 'Gender':['F','M']})
df1.merge(df2, on = 'Name', how = 'outer')

,Name,Age,Gender
0,San Zhang,20.0,NaN
1,Si Li,30.0,F
2,Wu Wang,NaN,M


In [4]:
df1 = pd.DataFrame({'df1_Name': ['San Zhang','Si Li'],'Age':[20,30]})
df2 = pd.DataFrame({'df2_Name':['Si Li','Wu Wang'], 'Gender':['F','M']})
df1.merge(df2, left_on = 'df1_Name', right_on = 'df2_Name', how = 'outer')

,df1_Name,Age,df2_Name,Gender
0,San Zhang,20.0,NaN,NaN
1,Si Li,30.0,Si Li,F
2,NaN,NaN,Wu Wang,M


In [12]:
df1 = pd.DataFrame({'Name':['San Zhang'], 'Grade':[70]})
df2 = pd.DataFrame({'Name':['San Zhang', 'San Zhang'], 'Grade':[80,90]})
df1.merge(df2, on = 'Name', how= 'left', suffixes=['_Chinese', '_Math'], validate='1:m')

,Name,Grade_Chinese,Grade_Math
0,San Zhang,70,80
1,San Zhang,70,90


In [13]:
df1 = pd.DataFrame({'Grade':[70]}, index = pd.Series(['San Zhang'], name = 'Name'))
df2 = pd.DataFrame({'Grade':[80]}, index = pd.Series(['San Zhang'], name = 'Name'))
df1.join(df2, how = 'left', lsuffix='_Chinese', rsuffix='_Math')

,Grade_Chinese,Grade_Math
Name,,
San Zhang,70,80


In [14]:
df1 = pd.DataFrame({'Name':['San Zhang', 'Si Li'], 'Age':[20,30]})
df2 = pd.DataFrame({'Name':['Wu Wang'], 'Age':[40]})
pd.concat([df1,df2])

,Name,Age
0,San Zhang,20
1,Si Li,30
0,Wu Wang,40


In [15]:
df2 = pd.DataFrame({'Grade':[80,90]})
df3 = pd.DataFrame({'Gender':['M','F']})
pd.concat([df1,df2,df3], 1)

C:\Users\puddi\AppData\Local\Temp/ipykernel_20332/3861152552.py:3: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  pd.concat([df1,df2,df3], 1)


,Name,Age,Grade,Gender
0,San Zhang,20,80,M
1,Si Li,30,90,F


In [16]:
df1 = pd.DataFrame({'Name':['San Zhang','Si Li'], 'Age':[20,21]})
df2 = pd.DataFrame({'Name':['Wu Wang'], 'Age':[21]})
pd.concat([df1,df2], keys=['one','two'])

Name  Age
one 0  San Zhang   20
    1      Si Li   21
two 0    Wu Wang   21

In [22]:
s = pd.Series(['Wu Wang', 21], index = df1.columns)
s

Name    Wu Wang
Age          21
dtype: object

In [23]:
df1.append(s, ignore_index=True)

,Name,Age
0,San Zhang,20
1,Si Li,21
2,Wu Wang,21


In [24]:
s = pd.Series([80,90])
df1.assign(Grade = s)
# 临时副本，df1中实际上还是没有'Grade'这列

,Name,Age,Grade
0,San Zhang,20,80
1,Si Li,21,90


## 类连接操作
### 比较`compare`
1. `df1.compare(df2)`比较两个表或者序列的不同处并将其汇总展示
   - 结果返回不同值所在的行列，如果相同则会被填充为缺失值NaN，其中other和self分别指代传入的参数表和被调用的表自身
   - `keep_shape=True`可以完整显示表中所有元素的比较情况
### 组合`combine`
1. 让两张表按照一定的规则进行组合，在进行规则比较时会自动进行列索引的对齐
2. 对于传入的函数而言，每一次操作中输入的参数是来自两个表的同名Series，依次传入的列是两个表列名的并集

In [26]:
df1 = pd.DataFrame({'Name':['San Zhang','Si Li','Wu Wang'],
                    'Age':[20,21,21],
                    'Class':['one','two','three']})
df2 = pd.DataFrame({'Name':['San Zhang','Li Si','Wu Wang'],
                    'Age':[20,22,21],
                    'Class':['one','two','Three']})
df1.compare(df2, keep_shape=True)

Name          Age        Class       
    self  other  self other   self  other
0    NaN    NaN   NaN   NaN    NaN    NaN
1  Si Li  Li Si  21.0  22.0    NaN    NaN
2    NaN    NaN   NaN   NaN  three  Three

In [27]:
def choose_min(s1,s2):
    s2 = s2.reindex_like(s1)
    res = s1.where(s1<s2, s2)
    res = res.mask(s1.isna())
    return res
df1 = pd.DataFrame({'A':[1,2],'B':[3,4],'C':[5,6]})
df2 = pd.DataFrame({'B':[5,6],'C':[7,8],'D':[9,10]}, index = [1,2])
df1.combine(df2, choose_min)

,A,B,C,D
0,NaN,NaN,NaN,NaN
1,NaN,4.0,6.0,NaN
2,NaN,NaN,NaN,NaN
